In [1]:
# Import the SVM model
from sklearn import svm
from sklearn.metrics import classification_report

# Import the required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from finta import TA
from pandas.tseries.offsets import DateOffset
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout

%matplotlib inline

In [3]:
# Import indicator dataframe
# Start with 1-minute... alghough we would like to attatch them all together soon.
df = pd.read_csv("../algotrader2/resources/aapl_1min_pivot_point_indicator_df.csv")

# df = pd.read_csv("../algotrader/resources/aapl_3min_pivot_point_indicator_df.csv")
# df.head()

# df = pd.read_csv("../algotrader/resources/aapl_1min_pivot_point_indicator_df.csv")
# df.head()

# Create our timestamp column as a datetime index, then save it as our index
df['timestamp'] = pd.to_datetime(df['timestamp'])

df.set_index('timestamp', inplace=True)

df.head()


,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:04:00+00:00,126.5001,126.880,126.4166,5733,126.820,513722,126.624892,-0.002516,126.87386,127.15340,...,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0,0
2023-01-03 15:05:00+00:00,126.8500,126.895,126.4400,3763,126.505,362900,126.625878,0.002766,126.80786,127.06139,...,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0,0
2023-01-03 15:06:00+00:00,126.5800,126.870,126.5300,2930,126.860,240388,126.664497,-0.002128,126.71786,126.94739,...,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0,0
2023-01-03 15:07:00+00:00,126.7100,126.870,126.5000,3189,126.580,289721,126.666956,0.001027,126.69186,126.87129,...,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0,0
2023-01-03 15:08:00+00:00,126.7066,126.760,126.5601,2861,126.705,255144,126.687038,-0.000027,126.66934,126.81495,...,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0,0


In [7]:
# X is everything except the signal column
X = df.drop('signal', axis=1)
# We should use the .shift() function so that our algorithm predicts the minute before realtime
# Drop the row with NaN values 
X = X.shift().dropna()
display(X.head())

# y is the signal column
y = df[("signal")]
display(y)

# Set start of training period
training_begin = X.index.min()
print(training_begin)

# Select ending period for the training data. Since we pulled a year's worth of data
# we will train on 9 months and then test with the rest
training_end = X.index.min() + DateOffset(months=9)
print(training_end)

# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
display(X_train_scaled)
display(X_test_scaled)

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,CCI,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:05:00+00:00,126.5001,126.880,126.4166,5733.0,126.820,513722.0,126.624892,-0.002516,126.87386,127.15340,...,-116.423590,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0
2023-01-03 15:06:00+00:00,126.8500,126.895,126.4400,3763.0,126.505,362900.0,126.625878,0.002766,126.80786,127.06139,...,-101.126762,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0
2023-01-03 15:07:00+00:00,126.5800,126.870,126.5300,2930.0,126.860,240388.0,126.664497,-0.002128,126.71786,126.94739,...,-99.309857,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0
2023-01-03 15:08:00+00:00,126.7100,126.870,126.5000,3189.0,126.580,289721.0,126.666956,0.001027,126.69186,126.87129,...,-88.964539,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0
2023-01-03 15:09:00+00:00,126.7066,126.760,126.5601,2861.0,126.705,255144.0,126.687038,-0.000027,126.66934,126.81495,...,-84.000283,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0


timestamp
2023-01-03 15:04:00+00:00    0
2023-01-03 15:05:00+00:00    0
2023-01-03 15:06:00+00:00    0
2023-01-03 15:07:00+00:00    0
2023-01-03 15:08:00+00:00    0
                            ..
2024-01-17 00:47:00+00:00    0
2024-01-17 00:52:00+00:00    0
2024-01-17 00:55:00+00:00    0
2024-01-17 00:57:00+00:00    0
2024-01-17 00:58:00+00:00    0
Name: signal, Length: 153682, dtype: int64

2023-01-03 15:05:00+00:00
2023-10-03 15:05:00+00:00


array([[-2.35124626, -2.33365604, -2.35202174, ..., -1.80656103,
         7.44654984, -1.12513333],
       [-2.33149852, -2.33280927, -2.3507014 , ..., -1.7587487 ,
         6.88368505, -1.12513333],
       [-2.34673685, -2.33422056, -2.3456232 , ..., -1.77349419,
         6.38717519, -1.12513333],
       ...,
       [ 0.18395335,  0.19028715,  0.18557785, ..., -0.68450998,
        -0.27235233,  1.14753163],
       [ 0.18762183,  0.18831135,  0.18558349, ..., -0.69605814,
        -0.27994579,  1.14753163],
       [ 0.18705745,  0.1926694 ,  0.18727058, ..., -0.67466796,
        -0.40165214,  1.14753163]])

array([[ 0.18705745,  0.1926694 ,  0.18727058, ..., -0.67466796,
        -0.40165214,  1.14753163],
       [ 0.19186034,  0.19028715,  0.19009181, ..., -0.64040263,
        -0.5609811 ,  1.14753163],
       [ 0.18827087,  0.19141618,  0.18896332, ..., -0.64296252,
        -0.64949377,  1.14753163],
       ...,
       [ 0.84032995,  0.84004305,  0.843487  , ...,  1.54445325,
        -0.47423821, -1.12513333],
       [ 0.83750804,  0.83609145,  0.84066577, ...,  0.16007704,
        -0.41855528, -1.12513333],
       [ 0.83807242,  0.83665596,  0.84123002, ..., -0.20120459,
        -0.49805655, -1.12513333]])

### SVM

In [8]:
# Create the classifier model
svm_model = svm.SVC()

# Fit the model to the data using X_train_scaled and y_train
# Fit is the 'training'
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
# This is predicting on the TRAINING portion of the scaled data. We will test it later with the TEST portion of the scaled data.
training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
training_signal_predictions[:10]

In [ ]:
# Evaluate the model's performance on the training data using a classification report
training_report = classification_report(y_train, training_signal_predictions)

# Display report
print(training_report)

In [ ]:
# Use the trained model to predict the trading signals for the testing data.
testing_signal_predictions = svm_model.predict(X_test_scaled)

In [ ]:
# Evaluate the model's ability to predict the trading signal for the testing data
testing_report = classification_report(y_test, testing_signal_predictions)

# Display the report
print(testing_report)

#### Visually Compare the Actual and Predicted Returns

In [ ]:
# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

predictions_df["predicted_signal"] = testing_signal_predictions

predictions_df["actual_returns"] = trading_df["actual_returns"]

predictions_df["trading_algorithm_returns"] = (
    predictions_df["actual_returns"] * predictions_df["predicted_signal"]
)

# Review the DataFrame
predictions_df.head()

In [ ]:
# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
(1 + predictions_df[["actual_returns", "trading_algorithm_returns"]]).cumprod().plot()

In [ ]:
# Evaluate the model using a classification report
training_report = classification_report(y_train, training_signal_predictions)

# Display report
print(training_report)